In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector

In [3]:
import getpass
db_user = getpass.getpass('Enter Username')
db_password = getpass.getpass('Enter password')

Enter Username········
Enter password········


In [4]:
config = {
    'user': db_user,
    'password': db_password,
    'host': 'localhost',
    'port': 3306,
    'database': 'election',
    'raise_on_warnings': True
}

In [5]:
db_connection = mysql.connector.connect(**config)

In [ ]:
cursor =  db_connection.cursor()
#Query 1

"""
The count of residents is based on the unique
V_ID_Num values in the Voting_Registry table for each city.
"""

# execute a SQL query using execute() method
q = ("""SELECT 
    P.City, P.State, 
    COUNT(DISTINCT VR.V_ID_Num) AS Residents 
FROM 
    Places P 
JOIN 
    Voting_Centers VC ON P.Address = VC.Address 
JOIN 
    Voting_Registry VR ON VC.Center_Name = VR.Center_Name
GROUP BY 
    P.City, P.State
HAVING 
    Residents > 0 
ORDER BY 
    Residents DESC;""") # replace with your table name

cursor.execute(q)
data = cursor.fetchall()
for i in data:
    print(i)


# close the cursor and connection
cursor.close()

In [ ]:
# create a cursor object using cursor() method
cursor = db_connection.cursor()

#query 2
"""
The given SQL query retrieves data from the "Folks," "Residencies," "Places," and "Email" tables.
"""

# execute a SQL query using execute() method
q = ("""SELECT 
    f.First_Name, f.Last_Name, p.City, e.email_address
FROM
    Folks f
JOIN
    Residencies r ON f.ID_Num = r.ID_Num
JOIN
    Places p ON r.Address = p.Address
JOIN
    Email e ON f.ID_Num = e.ID_Num
GROUP BY
    f.First_Name, f.Last_Name, p.City, e.email_address;
""") # replace with your table name

cursor.execute(q)
data = cursor.fetchall()
for i in data:
    print(i)


# close the cursor and connection
cursor.close()

In [ ]:
# create a cursor object using cursor() method
cursor = db_connection.cursor()

#query 3
"""
list each state along with its number of currently inhabited places, 
including states with no inhabited places, in increasing alphabetical order.
"""

# execute a SQL query using execute() method
q = ("""SELECT
    p.State,
    COUNT(DISTINCT p.Address) AS Num_Of_Places
FROM
    Places p
LEFT JOIN
    Residencies r ON p.Address = r.Address
GROUP BY
    p.State
ORDER BY
    p.State ASC;""") # replace with your table name

cursor.execute(q)
data = cursor.fetchall()
for i in data:
    print(i)


# close the cursor and connection
cursor.close()

In [ ]:
# create a cursor object using cursor() method
cursor = db_connection.cursor()

#query 4

"""
This query will return the distinct identifiers (V_ID_Num) of
the folks who registered at 'Springfield Center A' between 
'2023-05-10' and '2023-06-10'. 
"""
# get user input
center_name = input("Enter the center name: ")
start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = input("Enter the end date (YYYY-MM-DD): ")

# create a SQL query using placeholders
q = ("""SELECT DISTINCT V.V_ID_Num
FROM Voting_Registry V
WHERE V.Center_Name = %s AND V.Voting_Date BETWEEN %s AND %s;""")

# execute the query with the user's inputs
cursor.execute(q, (center_name, start_date, end_date))

data = cursor.fetchall()
for i in data:
    print(i)

# close the cursor and connection
cursor.close()


In [ ]:
# create a cursor object using cursor() method
cursor = db_connection.cursor()

#query 5

"""
Springfield A are megapolis centers, Springfield center B are the ones 
within 5 miles and Greenville has the excluded centers.
"""

# get user input
month = input("Enter the month (as an integer 1-12): ")
year = input("Enter the year: ")
center_names_included = input("Enter the center names to include, separated by commas: ").split(',')
center_names_excluded_prefix = input("Enter the prefix of center names to exclude: ")

center_names_included = [name.strip() for name in center_names_included]  # Strip extra spaces

# Create a SQL query using placeholders
q = ("""SELECT COUNT(DISTINCT VR.V_ID_Num) AS Unique_Registrations
FROM Voting_Registry VR
JOIN Voting_Centers VC ON VR.Center_Name = VC.Center_Name
JOIN Places P ON VC.Address = P.Address
WHERE MONTH(VR.Voting_Date) = %s AND YEAR(VR.Voting_Date) = %s
AND VC.Center_Name IN ({}) 
AND VC.Center_Name NOT LIKE %s;""".format(', '.join(['%s'] * len(center_names_included))))

# Prepare inputs
inputs = [month, year] + center_names_included + [center_names_excluded_prefix + '%']

# Execute the query with the user's inputs
cursor.execute(q, inputs)

data = cursor.fetchall()
for i in data:
    print(i)

# close the cursor and connection
cursor.close()


In [ ]:
# create a cursor object using cursor() method
cursor = db_connection.cursor()

#query 6

"""
Below is a possible MySQL query that could be used to find the most popular 
 voting center(s) (in terms of total number of registrations) in a given time 
 period among those in a random city from the loaded data.
"""

# get user input
start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = input("Enter the end date (YYYY-MM-DD): ")
limit = input("Enter the limit: ")

# create a SQL query using placeholders
q = ("""SELECT 
    VC.Center_Name, 
    COUNT(VR.V_ID_Num) AS Total_Registrations,
    P.City
FROM 
    Voting_Registry VR
    JOIN Voting_Centers VC ON VR.Center_Name = VC.Center_Name
    JOIN Places P ON VC.Address = P.Address
WHERE 
    VR.Voting_Date BETWEEN %s AND %s
GROUP BY 
    VC.Center_Name, P.City
ORDER BY 
    Total_Registrations DESC
LIMIT %s;""")

# execute the query with the user's inputs
cursor.execute(q, (start_date, end_date, int(limit)))

data = cursor.fetchall()
for i in data:
    print(i)

# close the cursor and connection
cursor.close()

In [ ]:
# create a cursor object using cursor() method
cursor = db_connection.cursor()

#query 7

"""
Combines multiple JOINs, a GROUP BY clause, 
and a HAVING clause to find unique folks that 
have valid registrations with every voting center
in a given state.
"""

# get user input
state = input("Enter the state: ")

# create a SQL query using placeholders
q = ("""SELECT 
    F.First_Name, F.Last_Name 
FROM 
    Folks F
JOIN 
    Voting_Registry VR ON F.ID_Num = VR.V_ID_Num
JOIN 
    Voting_Centers VC ON VR.Center_Name = VC.Center_Name
JOIN 
    Places P ON VC.Address = P.Address
WHERE 
    P.State = %s
GROUP BY 
    F.ID_Num, F.First_Name, F.Last_Name
HAVING 
    COUNT(DISTINCT VC.Center_Name) = (
        SELECT 
            COUNT(DISTINCT VC.Center_Name) 
        FROM 
            Voting_Centers VC
        JOIN 
            Places P ON VC.Address = P.Address
        WHERE 
            P.State = %s
    );""")

# execute the query with the user's inputs
cursor.execute(q, (state, state))

data = cursor.fetchall()
for i in data:
    print(i)

# close the cursor and connection
cursor.close()

In [ ]:
# create a cursor object using cursor() method
cursor = db_connection.cursor()

#query 9

"""
This function will now return the acronym of the center closest to the given
folk's residence that is operating at the given date and time. If no such 
center exists, it will return NULL.
"""

# get user input
folk_id = input("Enter the folk id: ")
given_date = input("Enter the date (YYYY-MM-DD): ")

# create a SQL query using placeholders
q = ("""SELECT GetClosestCenter(%s, %s);""")

# execute the query with the user's inputs
cursor.execute(q, (int(folk_id), given_date))

data = cursor.fetchall()
for i in data:
    print(i)

# close the cursor and connection
cursor.close()

In [6]:
# create a cursor object using cursor() method
cursor = db_connection.cursor()

#query 10

"""
This procedure will create a dynamic pivot table that shows the 
number of votes for each option of the given ballot, broken down
by voting center.
"""

# get user input
ballot_name = input("Enter the ballot name: ")

# create a SQL query using placeholders
q = "CALL BallotCrossTabulation(%s)"

# execute the query with the user's inputs
cursor.execute(q, [ballot_name])

data = cursor.fetchall()
for i in data:
    print(i)

# close the cursor and connection
cursor.close()

Enter the ballot name: Ballot 1


True

In [ ]:
cursor = db_connection.cursor()


# A clerk creating a new ballot:
ballot_name = input("Enter the new ballot name: ")
b_id_num = int(input("Enter the B_ID_NUM: "))
question_text = input("Enter the question text: ")
available_period = input("Enter the available period (format YYYY-MM-DD HH:MM:SS): ")
answers = input("Enter the answers, comma-separated: ")

q = """START TRANSACTION;
INSERT INTO Ballots (Ballot_Name, B_ID_NUM, Question_Text, Available_Period, Answers)
VALUES (%s, %s, %s, %s, %s);
COMMIT;"""
cursor.execute(q, (ballot_name, b_id_num, question_text, available_period, answers))


# close the cursor and connection
cursor.close()

In [ ]:
cursor = db_connection.cursor()


# A folk registering to vote for a ballot:
center_name = input("Enter the center name: ")
v_id_num = int(input("Enter the V_ID_NUM: "))
ballot_name = input("Enter the ballot name: ")
voting_date = input("Enter the voting date (format YYYY-MM-DD): ")

q = """START TRANSACTION;
INSERT INTO Voting_Registry (Center_Name, V_ID_Num, Ballot_Name, Voting_Date)
VALUES (%s, %s, %s, %s);
COMMIT;"""
cursor.execute(q, (center_name, v_id_num, ballot_name, voting_date))


# close the cursor and connection
cursor.close()

In [ ]:
cursor = db_connection.cursor()


# A clerk modifying the availability period of a current ballot:
new_available_period = input("Enter the new available period (format YYYY-MM-DD HH:MM:SS): ")
ballot_name = input("Enter the ballot name: ")

q = """START TRANSACTION;
UPDATE Ballots 
SET Available_Period = %s
WHERE Ballot_Name = %s;
COMMIT;"""
cursor.execute(q, (new_available_period, ballot_name))


# close the cursor and connection
cursor.close()

In [ ]:
cursor = db_connection.cursor()


# A voter casting a ballot while confirming a valid voting registration by a monitor:
v_id_num = int(input("Enter the V_ID_NUM: "))
ballot_name = input("Enter the ballot name: ")
voting_date = input("Enter the voting date (format YYYY-MM-DD): ")
question_text = input("Enter the question text: ")
answer = input("Enter the answer: ")
vote_time = input("Enter the vote time (format YYYY-MM-DD HH:MM:SS): ")

q = """START TRANSACTION;
-- Confirming a valid voting registration
SELECT * FROM Voting_Registry 
WHERE V_ID_Num = %s AND Ballot_Name = %s AND Voting_Date = %s;

-- If the above query returns a valid registration, proceed to cast a vote
INSERT INTO Cast_Votes (Voter_ID, Ballot_Name, Question_Text, Answer, Vote_Time)
VALUES (%s, %s, %s, %s, %s);
COMMIT;"""
cursor.execute(q, (v_id_num, ballot_name, voting_date, v_id_num, ballot_name, question_text, answer, vote_time))


# close the cursor and connection
cursor.close()

In [ ]:
cursor = db_connection.cursor()


# A staff removing a folk (and all their associated information):
id_num = int(input("Enter the ID_Num of the folk to be removed: "))

q = """START TRANSACTION;

-- Deleting all associated information
DELETE FROM Residencies WHERE ID_Num = %s;
DELETE FROM Email WHERE ID_Num = %s;

-- Get the Center_Name(s) for this folk
SET @centers = (SELECT GROUP_CONCAT(DISTINCT Center_Name) FROM Voting_Registry WHERE V_ID_Num = %s);

-- Delete from Voting_Centers
DELETE FROM Voting_Centers WHERE FIND_IN_SET(Center_Name, @centers) > 0;

-- Now you can safely delete from Voting_Registry
DELETE FROM Voting_Registry WHERE V_ID_Num = %s;

DELETE FROM Cast_Votes WHERE Voter_ID = %s;
DELETE FROM Election_Staff WHERE ES_ID_Num = %s;

-- Deleting the folk
DELETE FROM Folks WHERE ID_Num = %s;

COMMIT;"""
cursor.execute(q, (id_num, id_num, id_num, id_num, id_num, id_num, id_num))


# close the cursor and connection
cursor.close()